In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO

In [ ]:
rf = Roboflow(api_key="")
project = rf.workspace("first-708n1").project("aimbotminecraft")
version = project.version(1)
dataset = version.download("yolov11")

In [ ]:
# Load a pretrained YOLO11n model
model = YOLO("yolov11n.pt")

# Train the model on the COCO8 dataset for 100 epochs
train_results = model.train(
    data="aimbotMinecraft-1/data.yaml",  # Path to dataset configuration file
    epochs=100,  # Number of training epochs
    imgsz=640,  # Image size for training
    device="cuda",  # Device to run on (e.g., 'cpu', 0, [0,1,2,3])
)

In [ ]:
import pyautogui
import threading
import cv2
import mss
import numpy as np
from ultralytics import YOLO
import time
import keyboard
import pygetwindow as gw

In [ ]:
def get_window_rect(title_substring):
    for window in gw.getWindowsWithTitle(title_substring):
        if window.visible:
            rect = window.left, window.top, window.width, window.height
            return {
                "left": rect[0],
                "top": rect[1],
                "width": rect[2],
                "height": rect[3]
            }
    return None

def aim_loop(window_name):
    window_rect = get_window_rect(window_name)
    while True:
        with mss.mss() as sct:
            img = np.array(sct.grab(window_rect))
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

            results = model(img, verbose=False)[0]

            filtered_boxes = [box for box in results.boxes if box.conf[0] >= 0.5]

            boxes = sorted(filtered_boxes, key=lambda box: (box.xyxy[0][2] - box.xyxy[0][0]) * (
                    box.xyxy[0][3] - box.xyxy[0][1]), reverse=True)


            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes[i].xyxy[0])
                target_x = (x1 + x2) // 2
                target_y = (y1 + y2) // 2

                conf = float(boxes[i].conf[0])
                label = results.names[int(boxes[i].cls[0])]
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                if i == 0:
                    cv2.putText(img, f"{label} {conf:.2f} {target_x:.2f} { target_y:.2f}", (x1, y1 - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                    if keyboard.is_pressed('j'):
                        pyautogui.moveTo(target_x, target_y)
                        pyautogui.click(target_x, target_y)
                else:
                    cv2.putText(img, f"{label} {conf:.2f} {target_x:.2f} { target_y:.2f}", (x1, y1 - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


            cv2.imshow("YOLO Detection", img)
            if cv2.waitKey(1) == 27:
                break

    cv2.destroyAllWindows()

model = YOLO("best.pt")


temp = "Minecraft 1.20 - Multiplayer (3rd-party Server)"
print('Введите название окна для захвата: ', end='')
window_name = input()
threading.Thread(target=aim_loop(window_name), daemon=True).start()

while True:
    ...
